In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Multi-fidelity (MF) optimization

In most cases it is better to do many cheap evaluations of an approximation to the target function than it is to only optimize the target function. This example demonstrates the 'multi-fidelity' capabilities of xopt. 

We follow the implementation of multi-fidelity bayesian optimization used in botorch https://botorch.org/tutorials/multi_fidelity_bo to optimize the synthetic test function AugmentedHartmann https://botorch.org/api/test_functions.html.

The difference between normal Bayesian optimization and MF optimization is that we specify a 'cost' to making observations at a given fidelity. For this example we assume a base cost of 5 and a fidelity cost between 0-1. The algorithm should make many observations at lower fidelity relative to higher fidelity, lowering the total observation cost. 

NOTE: The cost parameter is required to be the LAST element of the variables list. Also this method is best suited for parallel observations of the test function.

In [2]:
# Import the class
from xopt import Xopt
from xopt.bayesian.generators.multi_fidelity import MultiFidelityGenerator
from xopt.bayesian.models.models import create_multi_fidelity_model
from botorch.test_functions.multi_fidelity import AugmentedHartmann
import logging
import os
SMOKE_TEST = os.environ.get('SMOKE_TEST')
logging.basicConfig(level=logging.INFO)

from botorch.acquisition.analytic import UpperConfidenceBound


The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [3]:
# Make a proper input file. 
import yaml
YAML = """
xopt: {output_path: null, verbose: true}

algorithm:
  name: multi_fidelity
  options:  
      processes: 4
      budget: 32
      verbose: True
      generator_options: {}

simulation: 
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate

vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]                          ## NOTE: THIS IS REQUIRED FOR MULTI-FIDELITY OPTIMIZATION
  objectives:
    y1: 'MINIMIZE'
  linked_variables: {}
  constants: {a: dummy_constant}

"""
config = yaml.safe_load(YAML)

In [4]:
if SMOKE_TEST:
    config['algorithm']['options']['budget'] = 3
    config['algorithm']['options']['processes'] = 1
    config['algorithm']['options']['generator_options']['num_restarts'] = 2
    config['algorithm']['options']['generator_options']['raw_samples'] = 2
    config['algorithm']['options']['generator_options']['base_acq'] = None

X = Xopt(config)
X

Loading config as dict.



            Xopt 
________________________________           
Version: 0.4.3+137.g5ac4672.dirty
Configured: True
Config as YAML:
xopt: {output_path: null, verbose: true, algorithm: cnsga}
algorithm:
  name: multi_fidelity
  function: xopt.bayesian.algorithms.multi_fidelity_optimize
  options:
    processes: 4
    budget: 32
    verbose: true
    generator_options: {}
    base_cost: 1.0
    custom_model: !!python/name:xopt.bayesian.models.models.create_multi_fidelity_model ''
    restart_file: null
    initial_x: null
simulation:
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate
  options: {extra_option: abc}
vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]
  objectives: {y1: MINIMIZE}
  linked_variables: {}
  constants: {a: dummy_constant}
  constraints: {}

# Run BayesOpt

In [5]:
# Pick one of these
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
#from concurrent.futures import ProcessPoolExecutor as PoolExecutor

#executor = PoolExecutor()
# This will also work. 
executor=None

In [6]:
# Change max generations
X.run(executor=executor)
results = X.results

INFO:xopt.bayesian.asynch_optimize:started running optimization with generator: <xopt.bayesian.generators.multi_fidelity.MultiFidelityGenerator object at 0x000001E0CFA1D0D0>
INFO:xopt.bayesian.asynch_optimize:starting optimization loop
INFO:xopt.bayesian.asynch_optimize:Submitting candidate 0: tensor([0.7905, 0.9636, 0.5610, 0.2393, 0.6601, 0.0253, 0.2710],
       dtype=torch.float64)
INFO:xopt.bayesian.asynch_optimize:total cost: 1.2709610164165497
INFO:xopt.bayesian.asynch_optimize:Submitting candidate 1: tensor([0.5729, 0.1048, 0.7607, 0.9095, 0.0789, 0.5335, 0.4867],
       dtype=torch.float64)
INFO:xopt.bayesian.asynch_optimize:total cost: 2.7576792538166046
INFO:xopt.bayesian.asynch_optimize:Submitting candidate 2: tensor([0.5447, 0.8570, 0.8708, 0.6038, 0.6233, 0.6443, 0.4854],
       dtype=torch.float64)
INFO:xopt.bayesian.asynch_optimize:total cost: 4.243029624223709
INFO:xopt.bayesian.asynch_optimize:Submitting candidate 3: tensor([0.2601, 0.8064, 0.7882, 0.1635, 0.7716, 0.78

Starting at time 2021-09-13T16:41:31-05:00


INFO:xopt.bayesian.asynch_optimize:Model creation time: 0.106 s
INFO:xopt.bayesian.asynch_optimize:Candidate generation time: 23.95 s
INFO:xopt.bayesian.asynch_optimize:Candidate(s): tensor([[0.4086, 0.8321, 0.7236, 0.0122, 0.7440, 0.5600, 0.0000],
        [0.3151, 0.7444, 0.6419, 0.0435, 0.8621, 0.9829, 0.0000],
        [0.0365, 0.6222, 0.7808, 0.0951, 0.6517, 0.7422, 0.0000],
        [0.0846, 0.9584, 0.8682, 0.0943, 0.9642, 0.7611, 0.0000]],
       dtype=torch.float64)
INFO:xopt.bayesian.asynch_optimize:Submitting candidate 4: tensor([0.4086, 0.8321, 0.7236, 0.0122, 0.7440, 0.5600, 0.0000],
       dtype=torch.float64)
INFO:xopt.bayesian.asynch_optimize:total cost: 6.600572168827057
INFO:xopt.bayesian.asynch_optimize:Submitting candidate 5: tensor([0.3151, 0.7444, 0.6419, 0.0435, 0.8621, 0.9829, 0.0000],
       dtype=torch.float64)
INFO:xopt.bayesian.asynch_optimize:total cost: 7.600572168827057
INFO:xopt.bayesian.asynch_optimize:Submitting candidate 6: tensor([0.0365, 0.6222, 0.7808,

### Get highest fidelity global optimum

In [7]:
# create generator object
gen = MultiFidelityGenerator(X.vocs)

In [8]:
# create model
model = create_multi_fidelity_model(results['variables'], results['corrected_objectives'], results['corrected_constraints'], X.vocs)

In [9]:
## NOTE: we want to get the minimum evaluated at the highest fidelity -> make sure to use get_recommendation
rec = gen.get_recommendation(model)
problem = AugmentedHartmann(negate=False)
problem(rec) ## NOTE: the correct global minimum is -3.32237

tensor([-3.3122], dtype=torch.float64)